In [15]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

def read_data_function(file_name):
    data_frame = pd.read_csv(file_name, sep = ",")
    data_frame = data_frame[["date", "open"]]
    data_frame["date"] = pd.to_datetime(data_frame["date"], format = "%Y-%m-%d")
    data_frame.index = data_frame.pop("date")
    #scaler = MinMaxScaler(feature_range = (-1, 1))
    #stock_price_val = data_frame["open"].values.reshape(-1,1)
    #data_frame["open"] = scaler.fit_transform(stock_price_val)


    return data_frame

In [16]:
import numpy as np
import pandas as pd
def data_prep_function(sequence_dataset,window_size):
    #sequence_dataset = [np.array(sequence_dataset[i*sliding_window_size: (i + 1) * sliding_window_size]) for i in range(len(sequence_dataset) // sliding_window_size)]
    #X = np.array([sequenc_dataset[i: i + num_steps] for i in range(len(sequence_dataset) - sliding_window_size)])
    #y = np.array([sequence_dataset[i + num_steps] for i in range(len(sequence_dataset) - sliding_window_size)])
    for i in range(1, window_size + 1):
        sequence_dataset[f"open-{i}"] = sequence_dataset["open"].shift(i)


    sequence_dataset.dropna(inplace = True)


    return sequence_dataset

In [17]:
from sklearn.preprocessing import MinMaxScaler
def matrix_formatting_and_normalize_dataset(sliding_window_dataset):
    sliding_window_dataset = sliding_window_dataset.to_numpy()
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(sliding_window_dataset)
    return normalized_data

In [18]:
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch
def feature_target_modeling(dataset):
    X = dataset[:, 1:]
    y = dataset[:,0]
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    y_train = np.reshape(y_train, (y_train.shape[0], 1))
    y_test = np.reshape(y_test, (y_test.shape[0], 1))
    X_train_tensor = torch.tensor(X_train, dtype = torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype = torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype = torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype = torch.float32)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_sub, val_sub = random_split(train_dataset, [train_size, val_size])
    val_loader = torch.utils.data.DataLoader(val_sub, batch_size=16, shuffle=False)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 16, shuffle = True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = False)
    return train_loader, val_loader, test_loader

In [28]:
import math
import torch.nn as nn
class CustomLSTM(nn.Module):
    def __init__(self, input_sz, hidden_sz, output_sz = 1):
        super().__init__()
        self.input_sz= input_sz
        self.hidden_size= hidden_sz
        self.W= nn.Parameter(torch.Tensor(input_sz, hidden_sz* 4))
        self.U= nn.Parameter(torch.Tensor(hidden_sz, hidden_sz* 4))
        self.bias= nn.Parameter(torch.Tensor(hidden_sz* 4))
        self.linear = nn.Linear(hidden_sz, output_sz)
        self.init_weights()

    def init_weights(self):
        stdv= 1.0/ math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, x,
                init_states=None):


        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_sz, _= x.size()
        hidden_seq= []
        if init_states is None:

            h_t, c_t= (torch.zeros(bs, self.hidden_size).to(x.device),
                            torch.zeros(bs, self.hidden_size).to(x.device))
        else:

             h_t, c_t= init_states

        HS= self.hidden_size
        for t in range(seq_sz):
                        x_t= x[:, t, :]
            # batch the computations into a single matrix multiplication
                        gates= x_t@ self.W+ h_t@ self.U+ self.bias
                        i_t, f_t, g_t, o_t= (
                            torch.sigmoid(gates[:, :HS]),# input
                            torch.sigmoid(gates[:, HS:HS*2]),# forget
                            torch.tanh(gates[:, HS*2:HS*3]),
                            torch.sigmoid(gates[:, HS*3:]),# output
                        )
                        c_t= f_t* c_t+ i_t* g_t
                        h_t= o_t* torch.tanh(c_t)
                        hidden_seq.append(h_t.unsqueeze(0))
        hidden_seq= torch.cat(hidden_seq, dim=0)
            # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq= hidden_seq.transpose(0, 1).contiguous()
        output = self.linear(hidden_seq[:, -1, :])
        return output

In [37]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def train_model(model: torch.nn.Module, train_dataloader : DataLoader,val_dataloader: DataLoader, epochs: int):
    model.train(True)
    optimizer = torch.optim.Adam(model.parameters())
    loss_function = torch.nn.MSELoss()
    loss_vectorize_train = np.zeros(epochs)
    loss_vectorize_val = np.zeros(epochs)

    for epoch in range(epochs):
        epoch_loss_train = 0
        for x_batch, y_batch in train_dataloader:
            optimizer.zero_grad()
            preds = model(x_batch)
            loss = loss_function(preds, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss_train += loss.item()
            loss_vectorize_train[epoch] = epoch_loss_train / len(train_dataloader)

        model.eval()

        with torch.no_grad():
            epoch_loss_val = 0
            for x_batch, y_batch in val_dataloader:
                preds_2 = model(x_batch)
                loss = loss_function(preds_2, y_batch)
                epoch_loss_val += loss.item()
                loss_vectorize_val[epoch] = epoch_loss_val / len(val_dataloader)

        print(f"Epoch {epoch}: Train loss {loss_vectorize_train[epoch]} || Val loss {loss_vectorize_val[epoch]}")



    return loss_vectorize_train, loss_vectorize_val

In [38]:
data = read_data_function("/content/all_stocks_5yr.csv")

In [39]:
window_data = data_prep_function(data, window_size = 10)

In [40]:
formated_data = matrix_formatting_and_normalize_dataset(window_data)

In [41]:
train_loader, val_loader, test_loader = feature_target_modeling(formated_data)

In [42]:
model = CustomLSTM(1, 6)

In [43]:
train_model(model, train_loader, val_loader,10)

Epoch 0: Train loss 8.901507526637503e-05 || Val loss 1.3462626371119658e-05
Epoch 1: Train loss 2.0444945395780862e-05 || Val loss 8.018274645387992e-06
Epoch 2: Train loss 1.5493497132401118e-05 || Val loss 1.5468163526670698e-05
Epoch 3: Train loss 1.341125785662444e-05 || Val loss 7.423701593574071e-06
Epoch 4: Train loss 1.310169851021806e-05 || Val loss 5.792445725423622e-06
Epoch 5: Train loss 1.1507120935756445e-05 || Val loss 8.245576470949792e-06
Epoch 6: Train loss 1.1183485701574818e-05 || Val loss 5.590535873645345e-06
Epoch 7: Train loss 1.0418459178574572e-05 || Val loss 6.736272205645746e-06
Epoch 8: Train loss 9.788952630761557e-06 || Val loss 5.400209990628549e-06
Epoch 9: Train loss 9.299756278356487e-06 || Val loss 7.3180170786737575e-06


(array([8.90150753e-05, 2.04449454e-05, 1.54934971e-05, 1.34112579e-05,
        1.31016985e-05, 1.15071209e-05, 1.11834857e-05, 1.04184592e-05,
        9.78895263e-06, 9.29975628e-06]),
 array([1.34626264e-05, 8.01827465e-06, 1.54681635e-05, 7.42370159e-06,
        5.79244573e-06, 8.24557647e-06, 5.59053587e-06, 6.73627221e-06,
        5.40020999e-06, 7.31801708e-06]))